# Creating Custom Feature Maps in Qiskit Aqua for <br>Quantum Support Vector Machines


Support vector machines (SVM) address the problem of supervised learning through the construction of a classifier. Havlicek *et al*. proposed two strategies to design a quantum SVM, namely the Quantum Kernel Estimator and the Quantum Variatonal Classifier. Both of these strategies use data that is provided classically and encodes it in the quantum state space through a quantum feature map.[1] The choice of which feature map to use is important and may depend on the given dataset we want to classify. In this tutorial, we show how to configure new feature maps in Aqua and explore their impact on the accuracy of the quantum classifier.

[1] Havlicek _et al_.  Nature **567**, 209-212 (2019). https://www.nature.com/articles/s41586-019-0980-2, https://arxiv.org/abs/1804.11326

Aqua provides several options for customizing the quantum feature map. In particular, there are four main parameters that can be used for model selection: the circuit depth, the data map function, the quantum gate set and the order of expansion. We will go through each of these parameters in this tutorial, but before getting started, let us review the main concepts of the quantum feature map discussed in [1].


### Review of the Quantum Feature Map


A quantum feature map nonlinearly maps classical datum **x** to a quantum state $|\Phi(\mathbf{x})\rangle\langle\Phi(\mathbf{x})|$, a vector in the Hilbert space of density matrices. Support vector machine classifiers find a hyperplane separating each vector $|\Phi(\mathbf{x}_i)\rangle\langle\Phi(\mathbf{x}_i)|$ depending on their label, supported by a reduced amount of vectors (the so-called support vectors). A key element of the feature map is not only the use of quantum state space as a feature space but also the way data are mapped into this high dimensional space.

Constructing feature maps based on quantum circuits that are hard to simulate classically is an important steps towards obtaining a quantum advantage over classical approaches. The authors of [1] proposed a family of feature maps that is conjectured to be hard to simulate classically and that can be implemented as short-depth circuits on near-term quantum devices.

$$ \mathcal{U}_{\Phi(\mathbf{x})}=\prod_d U_{\Phi(\mathbf{x})}H^{\otimes n},\ U_{\Phi(\mathbf{x})}=\exp\left(i\sum_{S\subseteq[1,n]}\phi_S(\mathbf{x})\prod_{k\in S} P_k\right) $$

The number of qubits $n$ in the quantum circuit is equal to the dimensionality of the classical data $\mathbf{x}$, which are encoded through the coefficients $\phi_S(\mathbf{x})$. The quantum circuit is composed of $d$ repeated layers of Hadamard gates interleaved with entangling blocks, which are expressed in terms of the Pauli gates $P_k \in \{\mathbb{1}_k, X_k, Y_k, Z_k \}$. The parameters $d$, $\phi_S$ and $P_k$ are mutable for both classification algorithms (Quantum Variational Classifier and Quantum Kernel Estimator) in Aqua. We note that the depth $d=1$ circuit considered in [1] can be efficiently simulated classically by uniform sampling, while the $d=2$ variant is conjectured to be hard to simulate classically.

<img src="images/uphi.PNG" width="400" />

The size of $S$ can be controled as well. We call the $r$-th order expansion, the feature map of this circuit family when $|S|\leq r$. In Aqua, the default is the second order expansion $|S|\leq 2$ used in [1], but can be increased. The greater the upper bound the more interactions will be taken into account. This gives $n$ singeltons $S=\{i\}$, and, depending on the connectivity graph of the quantum device, up to $\frac{n(n-1)}{2}$ couples to encode non-linear interactions.

Finally, we have a choice of the set of Pauli gates to use. Only contributions from $Z$ and $ZZ$ gates are considered in [1], as the corresponding $U_{\Phi(\mathbf{\mathbf{x}})}$ can be implemented efficiently, which is important for applications on NISQ devices.

### Programming the Quantum Feature Map

We will now see how to modify these four parameters (circuit depth, data map function, quantum gate set and expansion order) in Aqua. Documentation on the quantum feature maps can be found at https://qiskit.org/documentation/aqua/feature_maps.html. Two of the provided feature maps, `FirstOrderExpansion` and `SecondOrderExpansion`, allow modification of the depth and data map function, but not the quantum gate set. To configure and compare different feature maps, we will use synthetic data from `qsvm_datasets.py` that was generated by the `SecondOrderExpansion` feature map with default settings. As a result, we expect high classification accuracy when training the model with this same feature map. 


In [1]:
import numpy as np
import matplotlib.pyplot as plt
import functools

from qiskit import BasicAer
from qiskit.aqua import run_algorithm, QuantumInstance
from qiskit.aqua.components.feature_maps import SecondOrderExpansion, FirstOrderExpansion, PauliExpansion, self_product
from qiskit.aqua.algorithms import QSVM
from qsvm_datasets import *

In [2]:
# Generate synthetic training and test sets from the SecondOrderExpansion quantum feature map
feature_dim = 2
sample_Total, training_dataset, test_dataset, class_labels = ad_hoc_data(training_size=20, test_size=10,
                                                                         n=feature_dim, gap=0.3,
                                                                         PLOT_DATA=False)

# Using the statevector simulator
backend = BasicAer.get_backend('statevector_simulator')
random_seed = 10598

quantum_instance = QuantumInstance(backend, seed=random_seed, seed_transpiler=random_seed)

With this synthetic data, we will use the Quantum Kernel Estimator to test different feature maps. The first feature map we will test is the first order expansion, with circuit depth $d=2$, the default data map (discussed below), and a full connectivity graph. From there, we will explore more complex feature maps with higher order and nondiagonal expansions and custom functions to map the classical data.

#### 1. First Order Diagonal Expansion



A first order diagonal expansion is implemented with the `FirstOrderExpansion` feature map where $|S|=1$. The resulting circuit contains no interactions between features of the encoded data, and no entanglement. The feature map can take the following inputs:

- Number of qubits `feature_dimension`: equal to the dimensionality of the classical data, 
- Circuit depth $d$, `depth`: number of times to repeat the circuit 
- Entangler map to encode qubit connectivity: default is `entangler_map=None`, meaning we will use a pre-computed connectivity graph according to the next parameter 
- String parameter called `entanglement` with options `'full'` or `'linear'` to generate connectivity if it isn't provided in `entangler_map`: default value is `'full'`, meaning it will consider the connectivity graph to be complete and consider all $\frac{n(n-1)}{2}$ interactions
- Data map $\phi_S(\mathbf{x})$ that can encode non-linear connections in data: default form is  `data_map_func=self_product`, where `self_product` represents 

$$\phi_S:x\mapsto \Bigg\{\begin{array}{ll}
    x_i & \mbox{if}\ S=\{i\} \\
        (\pi-x_i)(\pi-x_j) & \mbox{if}\ S=\{i,j\}
    \end{array}$$.


While the connectivity graph is not specified for the separable `FirstOrderExpansion` feature map, it will be important for nonseparable cases such as `SecondOrderExpansion`.

In [3]:
# Generate the feature map
feature_map = FirstOrderExpansion(feature_dimension=feature_dim, depth=2)

# Run the Quantum Kernel Estimator and classify the test data
qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.7


We see that this feature map yields poor classification accuracy on data generated to be separable by the second order expansion.

#### 2. Second Order Diagonal Expansion

The `SecondOrderExpansion` feature map allows $|S|\leq2$, so interactions in the data will be encoded in the feature map, according to the connectivity graph and the classical data map. This option with default parameters is equivalent to the feature map described in [1].

In [4]:
feature_map = SecondOrderExpansion(feature_dimension=feature_dim, depth=2)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  1.0


As expected, the second order feature map yields high test accuracy on this dataset.

#### 3. Second Order Diagonal Expansion with Custom Data Map

Instead of using the default data map $\phi_S(\mathbf{x})$ in Aqua, we can encode the classical data using custom functions.

In [ ]:
def custom_data_map_func(x):
    """
    Define a function map from R^n to R.
    Args:
        x (np.ndarray): data
    Returns:
        double: the mapped value
    """
    coeff = x[0] if len(x) == 1 else \
        functools.reduce(lambda m, n: m * n, np.sin(np.pi - x))
    return coeff

The custom data map we created now represents the function 
$$\phi_S:x\mapsto \Bigg\{\begin{array}{ll}
    x_i & \mbox{if}\ S=\{i\} \\
        \sin(\pi-x_i)\sin(\pi-x_j) & \mbox{if}\ S=\{i,j\}
    \end{array}$$.

Let us now test this custom data map, defined in `custom_data_map.py`, on the synthetic dataset.

In [ ]:
from custom_data_map import custom_data_map_func

# entangler_map is a dictionary with source qubit index as keys and arrays of target qubit indices as values
entangler_map = [[0, 1]] # qubit 0 linked to qubit 1

feature_map = SecondOrderExpansion(feature_dimension=feature_dim, depth=2,
                                   data_map_func=custom_data_map_func,
                                   entangler_map=entangler_map)

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.8


We see that this choice for the data map function reduced the accuracy of the model.

#### 4. Second Order Pauli Expansion


For some applications, we may want to consider a more general form of the feature map. One way to generalize is to use `PauliExpansion` and specify a set of specific Pauli gates instead of only $Z$ gates. This feature map has the same parameters as `FirstOrderExpansion` and `SecondOrderExpansion` (namely, `depth`, `entangler_map`, `data_map_function`) and an additional `paulis` parameter to change the gate set. This parameter is a list of strings, each representing the desired Pauli gate. The default value is `['Z', 'ZZ']`, which is equivalent to `SecondOrderExpansion`.


Each string in `paulis` is implemented one at a time. A single character, for example `'Z'`, is implemented with one layer of single-qubit gates, while terms such as `'ZZ'` or `'XY'` are implemented with one layer of corresponding two-qubit entangling gates for each qubit pair available.

For example, the choice `paulis = ['Z', 'Y', 'ZZ']` generates a quantum feature map of the form 

$$\mathcal{U}_{\Phi(\mathbf{x})} = \left( \exp\left(i\sum_{jk} \phi_{\{j,k\}}(\mathbf{x}) Z_j \otimes Z_k\right) \, \exp\left(i\sum_{j} \phi_{\{j\}}(\mathbf{x}) Y_j\right) \, \exp\left(i\sum_j \phi_{\{j\}}(\mathbf{x}) Z_j\right) \, H^{\otimes n} \right)^d.$$ 

The depth $d=1$ version of this quantum circuit is shown below

<br>
<img src="images/depth1.PNG" width="400"/>
<br>

The circuit begins with a layer of Hadamard gates $H^{\otimes n}$, followed by a layer of $A$ gates and a layer of $B$ gates. The $A$ and $B$ gates are single-qubit rotations by the same set of angles $\phi_{\{j\}}(\mathbf{x})$ but around different axes: $B = e^{i\phi_{\{j\}}(\mathbf{x})Y_j}$ and $A = e^{i\phi_{\{j\}}(\mathbf{x})Z_j}$. The entangling $ZZ$ gate $e^{i \phi_{\{0,1\}}(\mathbf{x}) Z_0 Z_1}$ is parametrized by an angle $\phi_{\{0,1\}}(\mathbf{x})$ and can be implemented using two controlled-NOT gates and one $A'=e^{i\phi_{\{0,1\}}(x)Z_1}$ gate as shown in the figure.

As a comparison, `paulis = ['Z', 'ZZ']` creates the same circuit as above but without the $B$ gates, while `paulis = ['Z', 'YY']` creates a circuit with a layer of $A$ gates followed by a layer of entangling $YY$ gates.

Below, we test the `PauliExpansion` with `paulis=['Z', 'Y', 'ZZ']`. We don't expect good test accuracy with this model since this dataset was created to be separable by the `SecondOrderExpansion` feature map.

In [ ]:
feature_map = PauliExpansion(feature_dimension=feature_dim, depth=2, paulis = ['Z','Y','ZZ'])

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

testing success ratio:  0.35


#### 5. Third Order Pauli Expansion with Custom Data Map

One should note that `PauliExpansion` allows third order or more expansions, for example `paulis = ['Z', 'ZZ', 'ZZZ']`. Assuming the data has dimensionality of at least three and we have access to three qubits, this choice for `paulis` generates a feature map according to the previously mentioned rule, with $|S|\leq 3$. 

For example, suppose we want to classify three-dimensional data using a third order expansion, a custom data map, and a circuit depth of 𝑑=2. We can do this with the following code in Aqua.

In [ ]:
feature_dim = 3
sample_Total_b, training_dataset_b, test_dataset_b, class_labels = ad_hoc_data(training_size=20, test_size=10, 
                                                                     n=feature_dim, gap=0.3, 
                                                                     PLOT_DATA=False)

In [ ]:
feature_map = PauliExpansion(feature_dimension=feature_dim, depth=2, paulis = ['Y','Z','ZZ','ZZZ'])

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset_b, test_dataset=test_dataset_b)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

The qubit connectivity is `'full'` by default, so this circuit will contain a layer of $B$ gates parametrized by $\phi_{\{j\}}(\mathbf x)$, a layer of $A$ gates parametrized by $\phi_{\{j\}}(\mathbf x)$, three $ZZ$ entanglers, one for each pair of qubits $(0,1),\ (1,2),\ (0,2)$, and finally a $ZZZ$ entangler $e^{i\phi_{\{0,1,2 \}}(x)Z_0Z_1Z_2}$. 

### Building New Feature Maps


We saw how to generate feature maps from the circuit family described in [1]. To explore new circuit families, we can create a new class implementing the class `FeatureMap`, and its method `construct_circuit`. As long as our custom feature map class has a working constructor and implementation of the method `construct_circuit`, it will be pluggable in any Aqua component requiring a feature map.

As an example, below we show a general custom feature map class, taking the circuit construction algorithm (the core of the feature map, the way it's generating the circuit), and a list of necessary arguments.

In [ ]:
"""
This module contains the definition of a base class for
feature map. Several types of commonly used approaches.
"""

import numpy as np
from inspect import signature
import logging
logger = logging.getLogger(__name__)

from qiskit import QuantumCircuit, QuantumRegister
from qiskit.aqua.components.feature_maps import FeatureMap

class CustomExpansion(FeatureMap):
    """
    Mapping data using a custom feature map.
    """

    CONFIGURATION = {
        'name': 'CustomExpansion',
        'description': 'Custom expansion for feature map (any order)',
        'input_schema': {
            '$schema': 'http://json-schema.org/schema#',
            'id': 'Custom_Expansion_schema',
            'type': 'object',
            'properties': {'feature_param': {'type': ['array']}},
            'additionalProperties': False
        }
    }

    def __init__(self, feature_dimension, constructor_function, feature_param):
        """Constructor.

        Args:
            num_qubits (int): number of qubits
            constructor_function (fun): a function that takes as parameters
            a datum x, a QuantumRegister qr, a boolean inverse and
            all other parameters needed from feature_param
            feature_param (list): the list of parameters needed to generate
            the circuit, that won't change depending on the data given
            (such as the data map function or other).
        """
        self.validate(locals())
        super().__init__()
        self._num_qubits = self._feature_dimension = feature_dimension
        sig = signature(constructor_function)
        if len(sig.parameters) != len(feature_param)+3:
            raise ValueError("The constructor_function given don't match the parameters given.\n" +
                             "Make sure it takes, in this order, the datum x, the QuantumRegister qr, the Boolean\n" +
                             " inverse and all the parameters provided in feature_param")
        self._constructor_function = constructor_function
        self._feature_param = feature_param
    
    def construct_circuit(self, x, qr=None, inverse=False):
        """
        Construct the circuit based on given data and according to the function provided at instantiation.

        Args:
            x (numpy.ndarray): 1-D to-be-transformed data.
            qr (QauntumRegister): the QuantumRegister object for the circuit, if None,
                                  generate new registers with name q.
            inverse (bool): whether or not to invert the circuit

        Returns:
            qc (QuantumCircuit): a quantum circuit to transform data x.
        """
        if not isinstance(x, np.ndarray):
            raise TypeError("x must be numpy array.")
        if x.ndim != 1:
            raise ValueError("x must be 1-D array.")
        if x.shape[0] != self._num_qubits:
            raise ValueError("number of qubits and data dimension must be the same.")
        if qr is None:
            qr = QuantumRegister(self._num_qubits, name='q')
        qc = self._constructor_function(x, qr, inverse, *self._feature_param)
        return qc


With this general class, we can use whatever rule we want to construct the circuit of our custom feature map. It can have the parameters we want, use the gates we want etc... We test it with a mock constructor function that creates a feature map consisting of successive layers of $R_X$ gates and $ZZ$ gates.

In [ ]:
def constructor_function(x, qr, inverse=False, depth=2, entangler_map=None):
    """A mock constructor function to test the CustomExpansion class.
    
    Args:
        x (numpy.ndarray): 1D to-be-transformed data
        qr (QuantumRegister)
        inverse (bool): whether or not to invert the circuit
        depth (int): number of times to repeat circuit
        entangler_map (dict): describe the connectivity of qubits
    
    Returns:
        qc (QuantumCircuit): layers of Rx gates interleaved with ZZ gates
    """
    
    if entangler_map is None:
        entangler_map = {i: [j for j in range(i, len(x)) if j != i] for i in range(len(x) - 1)}
    
    qc = QuantumCircuit(qr)

    for _ in range(depth):
        for i in range(len(x)):
            qc.rx(x[i], qr[i])
        for source in entangler_map:
            for target in entangler_map[source]:
                qc.cx(qr[source], qr[target])
                qc.u1(x[source] * x[target], qr[target])
                qc.cx(qr[source], qr[target])
    return qc

Below, we test our custom feature map on the synthetic dataset. Its parameters are `num_qubits`, our mock constructor function and a list containing the parameters. Now, using `feature_map` will create circuits using our constructor function, with the parameters given in the list.

In [ ]:
from custom_feature_map import CustomExpansion
from mock_constructor import constructor_function

feature_map = CustomExpansion(feature_dimension=2, constructor_function=constructor_function, feature_param=[2,None])

qsvm = QSVM(feature_map=feature_map, training_dataset=training_dataset, test_dataset=test_dataset)

result = qsvm.run(quantum_instance)
print("testing success ratio: ", result['testing_accuracy'])

Whether we want to use easily-configurable existing feature maps, or create entirely new custom feature maps within Aqua's pluggable interface, the tools available in Aqua enable users to further explore the applications of quantum support vector machines on near-term quantum devices.